# Build an MNIST Classifier Pipeline using Kubeflow and SageMaker

The `mnist-classification-pipeline.py` sample runs a pipeline to train a classficiation model using Kmeans with MNIST dataset on Sagemaker.

We will have all required steps here and for other details like how to get source data, please check [documentation](https://github.com/kubeflow/pipelines/tree/master/samples/contrib/aws-samples/mnist-kmeans-sagemaker).


This sample is based on the [Train a Model with a Built-in Algorithm and Deploy it](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1.html).

The sample trains and deploy a model based on the [MNIST dataset](http://www.deeplearning.net/tutorial/gettingstarted.html).



## Install AWS Python SDK (`boto3`)

In [1]:
!pip install boto3

You are using pip version 19.0.1, however version 20.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Install Kubeflow Pipelines SDK

In [2]:
!pip install https://storage.googleapis.com/ml-pipeline/release/0.1.29/kfp.tar.gz --upgrade

  Using cached https://storage.googleapis.com/ml-pipeline/release/0.1.29/kfp.tar.gz
  Stored in directory: /tmp/pip-ephem-wheel-cache-9rwckmwq/wheels/81/b7/33/00ef9dd992b13add014c4875a2c130d9d70288127a793c4af6
Successfully built kfp
  Found existing installation: kfp 0.1.29
    Uninstalling kfp-0.1.29:
      Successfully uninstalled kfp-0.1.29
You are using pip version 19.0.1, however version 20.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
# Restart the kernel to pick up pip installed libraries
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [1]:
import boto3

AWS_REGION_AS_SLIST=!curl -s http://169.254.169.254/latest/meta-data/placement/availability-zone | sed 's/\(.*\)[a-z]/\1/'
AWS_REGION = AWS_REGION_AS_SLIST.s
print('Region: {}'.format(AWS_REGION))

AWS_ACCOUNT_ID=boto3.client('sts').get_caller_identity().get('Account')
print('Account ID: {}'.format(AWS_ACCOUNT_ID))

S3_BUCKET='sagemaker-{}-{}'.format(AWS_REGION, AWS_ACCOUNT_ID)
print('S3 Bucket: {}'.format(S3_BUCKET))

Region: us-west-2
Account ID: 250107111215
S3 Bucket: sagemaker-us-west-2-250107111215


## Copy `data` and `valid_data.csv` into your S3 bucket.

In [2]:
!aws s3 cp s3://kubeflow-pipeline-data/mnist_kmeans_example/data s3://$S3_BUCKET/mnist_kmeans_example/data
!aws s3 cp s3://kubeflow-pipeline-data/mnist_kmeans_example/input/valid_data.csv s3://$S3_BUCKET/mnist_kmeans_example/input/

copy: s3://kubeflow-pipeline-data/mnist_kmeans_example/data to s3://sagemaker-us-west-2-250107111215/mnist_kmeans_example/data
copy: s3://kubeflow-pipeline-data/mnist_kmeans_example/input/valid_data.csv to s3://sagemaker-us-west-2-250107111215/mnist_kmeans_example/input/valid_data.csv


# Build Pipeline

# 1. Run the following command to load Kubeflow Pipelines SDK

In [3]:
import kfp
from kfp import components
from kfp import dsl
from kfp.aws import use_aws_secret

# 2. Load reusable sagemaker components.

In [4]:
sagemaker_train_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/0ad6c28d32e2e790e6a129b7eb1de8ec59c1d45f/components/aws/sagemaker/train/component.yaml')
sagemaker_model_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/0ad6c28d32e2e790e6a129b7eb1de8ec59c1d45f/components/aws/sagemaker/model/component.yaml')
sagemaker_deploy_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/0ad6c28d32e2e790e6a129b7eb1de8ec59c1d45f/components/aws/sagemaker/deploy/component.yaml')
sagemaker_batch_transform_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/0ad6c28d32e2e790e6a129b7eb1de8ec59c1d45f/components/aws/sagemaker/batch_transform/component.yaml')

# 3. Create pipeline. 

We will create a training job first. Once training job is done, it will persist trained model to S3. 

Then a job will be kicked off to create a `Model` manifest in Sagemaker. 

With this model, batch transformation job can use it to predict on other datasets, prediction service can create an endpoint using it.


> Note: remember to use your **role_arn** to successfully run the job.

> Note: If you use a different region, please replace `us-west-2` with your region. 

> Note: ECR Images for k-means algorithm

|Region| ECR Image|
|------|----------|
|us-west-1|632365934929.dkr.ecr.us-west-1.amazonaws.com|
|us-west-2|174872318107.dkr.ecr.us-west-2.amazonaws.com|
|us-east-1|382416733822.dkr.ecr.us-east-1.amazonaws.com|
|us-east-2|404615174143.dkr.ecr.us-east-2.amazonaws.com|
|us-gov-west-1|226302683700.dkr.ecr.us-gov-west-1.amazonaws.com|
|ap-east-1|286214385809.dkr.ecr.ap-east-1.amazonaws.com|
|ap-northeast-1|351501993468.dkr.ecr.ap-northeast-1.amazonaws.com|
|ap-northeast-2|835164637446.dkr.ecr.ap-northeast-2.amazonaws.com|
|ap-south-1|991648021394.dkr.ecr.ap-south-1.amazonaws.com|
|ap-southeast-1|475088953585.dkr.ecr.ap-southeast-1.amazonaws.com|
|ap-southeast-2|712309505854.dkr.ecr.ap-southeast-2.amazonaws.com|
|ca-central-1|469771592824.dkr.ecr.ca-central-1.amazonaws.com|
|eu-central-1|664544806723.dkr.ecr.eu-central-1.amazonaws.com|
|eu-north-1|669576153137.dkr.ecr.eu-north-1.amazonaws.com|
|eu-west-1|438346466558.dkr.ecr.eu-west-1.amazonaws.com|
|eu-west-2|644912444149.dkr.ecr.eu-west-2.amazonaws.com|
|eu-west-3|749696950732.dkr.ecr.eu-west-3.amazonaws.com|
|me-south-1|249704162688.dkr.ecr.me-south-1.amazonaws.com|
|sa-east-1|855470959533.dkr.ecr.sa-east-1.amazonaws.com|

In [5]:
SAGEMAKER_ROLE_ARN='arn:aws:iam::{}:role/TeamRole'.format(AWS_ACCOUNT_ID)

# Configure your s3 bucket.
S3_PIPELINE_PATH='s3://{}/mnist_kmeans_example'.format(S3_BUCKET)

# TODO:  Implement the other region checks
if AWS_REGION == 'us-west-2':
    AWS_ECR_REGISTRY='174872318107.dkr.ecr.us-west-2.amazonaws.com'

if AWS_REGION == 'us-east-1':
    AWS_ECR_REGISTRY='382416733822.dkr.ecr.us-east-1.amazonaws.com'

    
@dsl.pipeline(
    name='MNIST Classification pipeline',
    description='MNIST Classification using KMEANS in SageMaker'
)
def mnist_classification(region=AWS_REGION,
    image='{}/kmeans:1'.format(AWS_ECR_REGISTRY),
    dataset_path=S3_PIPELINE_PATH + '/data',
    instance_type='ml.c4.8xlarge',
    instance_count='2',
    volume_size='50',
    model_output_path=S3_PIPELINE_PATH + '/model',
    batch_transform_input=S3_PIPELINE_PATH + '/input',
    batch_transform_ouput=S3_PIPELINE_PATH + '/output',
    role_arn=SAGEMAKER_ROLE_ARN
    ):

    training = sagemaker_train_op(
        region=region,
        image=image,
        instance_type=instance_type,
        instance_count=instance_count,
        volume_size=volume_size,
        dataset_path=dataset_path,
        model_artifact_path=model_output_path,
        role=role_arn,
    ).apply(use_aws_secret('aws-secret', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY'))

    create_model = sagemaker_model_op(
        region=region,
        image=image,
        model_artifact_url=training.outputs['model_artifact_url'],
        model_name=training.outputs['job_name'],
        role=role_arn
    ).apply(use_aws_secret('aws-secret', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY'))

    prediction = sagemaker_deploy_op(
        region=region,
        model_name=create_model.output
    ).apply(use_aws_secret('aws-secret', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY'))

    batch_transform = sagemaker_batch_transform_op(
        region=region,
        model_name=create_model.output,
        input_location=batch_transform_input,
        output_location=batch_transform_ouput
    ).apply(use_aws_secret('aws-secret', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY'))

# 4. Compile your pipeline

In [6]:
kfp.compiler.Compiler().compile(mnist_classification, 'mnist-classification-pipeline.zip')

In [7]:
!ls -al ./mnist-classification-pipeline.zip

-rw-r--r-- 1 root users 1498 Aug 22 23:37 ./mnist-classification-pipeline.zip


In [8]:
!unzip -o ./mnist-classification-pipeline.zip

Archive:  ./mnist-classification-pipeline.zip
  inflating: pipeline.yaml           


In [9]:
!cat pipeline.yaml

apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  annotations:
    pipelines.kubeflow.org/pipeline_spec: '{"description": "MNIST Classification using
      KMEANS in SageMaker", "inputs": [{"default": "us-west-2", "name": "region"},
      {"default": "174872318107.dkr.ecr.us-west-2.amazonaws.com/kmeans:1", "name":
      "image"}, {"default": "s3://sagemaker-us-west-2-250107111215/mnist_kmeans_example/data",
      "name": "dataset_path"}, {"default": "ml.c4.8xlarge", "name": "instance_type"},
      {"default": "2", "name": "instance_count"}, {"default": "50", "name": "volume_size"},
      {"default": "s3://sagemaker-us-west-2-250107111215/mnist_kmeans_example/model",
      "name": "model_output_path"}, {"default": "s3://sagemaker-us-west-2-250107111215/mnist_kmeans_example/input",
      "name": "batch_transform_input"}, {"default": "s3://sagemaker-us-west-2-250107111215/mnist_kmeans_example/output",
      "name": "batch_transform_ouput"}, {"default": "arn:aws:iam::250107111215

# 5. Deploy your pipeline

In [10]:
client = kfp.Client()
aws_experiment = client.create_experiment(name='aws')
my_run = client.run_pipeline(aws_experiment.id, 'mnist-classification-pipeline', 
  'mnist-classification-pipeline.zip')

## Training

_Note:  The above training job may take 5-10 minutes.  Please be patient._

In the meantime, open the SageMaker Console to monitor the progress of your training job.

![SageMaker Training Job Console](img/sagemaker-training-job-console.png)

## Get the Name of the Deployed Prediction Endpoint
First, we need to get the endpoint name of our newly-deployed SageMaker Prediction Endpoint.

Open AWS console and enter SageMaker service, find the endpoint name as the following picture shows.

![download-pipeline](images/sm-endpoint.jpg)

# Make a Prediction

# _YOU MUST COPY/PASTE THE `ENDPOINT_NAME` BEFORE CONTINUING_
Make sure to include preserve the single-quotes as shown below.

In [12]:
import pickle, gzip, numpy, urllib.request, json
from urllib.parse import urlparse
import json
import io
import boto3

#################################
#################################
# Replace ENDPOINT_NAME with the endpoint name in the SageMaker console.
# Surround with single quotes.
ENDPOINT_NAME= 'Endpoint-20200822233740-UR9D'
#################################
#################################

# Load the dataset
urllib.request.urlretrieve("http://deeplearning.net/data/mnist/mnist.pkl.gz", "mnist.pkl.gz")
with gzip.open('mnist.pkl.gz', 'rb') as f:
    train_set, valid_set, test_set = pickle.load(f, encoding='latin1')

# Simple function to create a csv from our numpy array
def np2csv(arr):
    csv = io.BytesIO()
    numpy.savetxt(csv, arr, delimiter=',', fmt='%g')
    return csv.getvalue().decode().rstrip()

runtime = boto3.Session(region_name=AWS_REGION).client('sagemaker-runtime')

payload = np2csv(train_set[0][30:31])

response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                   ContentType='text/csv',
                                   Body=payload)
result = json.loads(response['Body'].read().decode())
print(result)

{'predictions': [{'distance_to_cluster': 7.2732768058776855, 'closest_cluster': 3.0}]}


## Clean up

Go to Sagemaker console and delete `endpoint` and `model`.